# Data Cleaning

### Load dataset and check 

In [1]:
import pandas as pd
import json
import numpy as np
import math

In [2]:
leboncoin_ads=pd.read_json("C:/Users/Annek/Documents/semestre 1/ML avec python/leboncoin/Leboncoin_Predicting_Price/leboncoin_ads.json")
leboncoin_ads.head()

,list_id,first_publication_date,expiration_date,index_date,status,category_id,category_name,subject,body,brand,...,price_cents,images,attributes,location,owner,options,has_phone,similar,counters,is_boosted
0,2909625822,2024-12-28 12:58:33,2025-02-26 12:58:33,2024-12-28 12:58:33,active,2,Voitures,Renault Mégane 3 1.5 Dci 105cv tomtom édition,,leboncoin,...,350000,{'thumb_url': 'https://img.leboncoin.fr/api/v1...,"[{'key': 'profile_picture_url', 'value': 'http...","{'country_id': 'FR', 'region_id': '16', 'regio...","{'store_id': '15939704', 'user_id': '7775eb0a-...","{'has_option': False, 'booster': False, 'photo...",True,NaN,{},NaN
1,2909625799,2024-12-28 12:58:33,2025-02-26 12:58:33,2024-12-28 12:58:33,active,2,Voitures,Hyundai i20,,leboncoin,...,1070000,{'thumb_url': 'https://img.leboncoin.fr/api/v1...,"[{'key': 'rating_score', 'value': '1', 'values...","{'country_id': 'FR', 'region_id': '18', 'regio...","{'store_id': '49507330', 'user_id': 'eb077456-...","{'has_option': False, 'booster': False, 'photo...",False,NaN,{},NaN
2,2909625761,2024-12-28 12:58:28,2025-02-26 12:58:28,2024-12-28 12:58:28,active,2,Voitures,Nissan Pixo,,leboncoin,...,299000,{'thumb_url': 'https://img.leboncoin.fr/api/v1...,"[{'key': 'profile_picture_url', 'value': 'http...","{'country_id': 'FR', 'region_id': '21', 'regio...","{'store_id': '62118662', 'user_id': '4c3349cf-...","{'has_option': False, 'booster': False, 'photo...",False,NaN,{},NaN
3,2909625747,2024-12-28 12:58:28,2025-02-26 12:58:28,2024-12-28 12:58:28,active,2,Voitures,Vends Nissan Qashqai Blue Connect Edition,,leboncoin,...,1100000,{'thumb_url': 'https://img.leboncoin.fr/api/v1...,"[{'key': 'profile_picture_url', 'value': 'http...","{'country_id': 'FR', 'region_id': '21', 'regio...","{'store_id': '26209241', 'user_id': 'cec7b0a3-...","{'has_option': False, 'booster': False, 'photo...",False,NaN,{},NaN
4,2909625688,2024-12-28 12:58:28,2025-02-26 12:58:28,2024-12-28 12:58:28,active,2,Voitures,Laguna III GT 4control,,leboncoin,...,940000,{'thumb_url': 'https://img.leboncoin.fr/api/v1...,"[{'key': 'rating_score', 'value': '0.98', 'val...","{'country_id': 'FR', 'region_id': '11', 'regio...","{'store_id': '19468680', 'user_id': '40ac2869-...","{'has_option': False, 'booster': False, 'photo...",False,NaN,{},NaN


In [3]:
leboncoin_ads.shape

(35, 23)

In [4]:
leboncoin_ads.columns

Index(['list_id', 'first_publication_date', 'expiration_date', 'index_date',
       'status', 'category_id', 'category_name', 'subject', 'body', 'brand',
       'ad_type', 'url', 'price', 'price_cents', 'images', 'attributes',
       'location', 'owner', 'options', 'has_phone', 'similar', 'counters',
       'is_boosted'],
      dtype='object')

In [5]:
#compter le nombre de modalités par column
for column_name in leboncoin_ads.columns:
    try:
        print(column_name," : ",len(set(leboncoin_ads[column_name])))
    except TypeError as e:
        print(column_name, f"Error: {e}")
        pass

list_id  :  35
first_publication_date  :  24
expiration_date  :  24
index_date  :  14
status  :  1
category_id  :  1
category_name  :  1
subject  :  34
body  :  1
brand  :  1
ad_type  :  1
url  :  35
price Error: unhashable type: 'list'
price_cents  :  34
images Error: unhashable type: 'dict'
attributes Error: unhashable type: 'list'
location Error: unhashable type: 'dict'
owner Error: unhashable type: 'dict'
options Error: unhashable type: 'dict'
has_phone  :  2
similar  :  35
counters Error: unhashable type: 'dict'
is_boosted  :  25


### Clean columns

First selection of variables

In [6]:

#Sélectionner variables pertinentes:
# - Index_date=first_publication_date (retirer doublons)
# - Retirer variables avec une seule modalité
# - Retirer "subject" car les infos sont déjà présentes dans "attributes"
# -"similar" ne contient que des NaN
# -"counters" ne contient que des dictionnaires vides
# -retirer url de l'annonce
# -price_cents=price*100
leboncoin_ads_filtered=leboncoin_ads[['list_id', 'first_publication_date', 'expiration_date', 
       'ad_type', 'price', 'images', 'attributes',
       'location', 'owner', 'options', 'has_phone',
       'is_boosted']]

Handling variables containing lists and dictionnairies

In [7]:
#Price: retrieve value from the list
leboncoin_ads_filtered.loc[:, "price"]=leboncoin_ads_filtered["price"].apply(lambda prix: prix[0])

In [8]:
#Shatter attributes:
ploo=pd.DataFrame()
attributs=pd.DataFrame()
for row_num in range( leboncoin_ads_filtered.shape[0]):
    ploo_col=pd.DataFrame()
    for column_name in ["location",'owner','options']:
    # Extract 'attributes' column as a DataFrame
        temporary_df=pd.DataFrame(leboncoin_ads_filtered.loc[row_num,column_name].items()).set_index(0).transpose()
        ploo_col = pd.concat([ploo_col,temporary_df],axis=1)
        ploo_col["list_id"]=leboncoin_ads_filtered.loc[row_num,"list_id"]
    ploo=pd.concat([ploo,ploo_col],axis=0,ignore_index=True)
    temporary_df=pd.DataFrame(leboncoin_ads_filtered.loc[row_num,"attributes"]).set_index("key").transpose()
    temporary_df=temporary_df[temporary_df.index=="value_label"]
    temporary_df["list_id"]=leboncoin_ads_filtered.loc[row_num,"list_id"]
    attributs=pd.concat([attributs,temporary_df],axis=0,ignore_index=True)
leboncoin_ads_filtered_extended=leboncoin_ads_filtered.merge(pd.concat([attributs,ploo],axis=1).T.reset_index().drop_duplicates(subset=["index"]).set_index('index').T,
                                                             how='outer',on='list_id')


In [9]:
print("Taille:",leboncoin_ads_filtered_extended.shape,"\ncolumns: ",leboncoin_ads_filtered_extended.columns)

Taille: (35, 81) 
columns:  Index(['list_id', 'first_publication_date', 'expiration_date', 'ad_type',
       'price', 'images', 'attributes', 'location', 'owner', 'options',
       'has_phone', 'is_boosted', 'profile_picture_url', 'brand', 'model',
       'u_car_brand', 'u_car_model', 'regdate', 'mileage', 'fuel', 'gearbox',
       'doors', 'seats', 'vehicle_technical_inspection_a', 'issuance_date',
       'vehicle_damage', 'vehicle_type', 'vehicle_upholstery',
       'vehicle_specifications', 'vehicle_interior_specs', 'vehicule_color',
       'critair', 'horsepower', 'horse_power_din', 'vehicle_vsp', 'is_import',
       'vehicle_is_eligible_p2p', 'car_rotation_delay', 'argus_object_id',
       'licence_plate_available', 'rating_score', 'rating_count',
       'ad_warranty_type', 'car_price_min', 'car_price_max',
       'vehicle_history_report_public_url', 'vehicle_euro_emissions_standard',
       'activity_sector', 'recent_used_vehicle',
       'vehicle_history_report_status', 'spare_p

Clean columns and some feature engineering

In [10]:
#is_boosted : remplacer les NaN par 0
leboncoin_ads_filtered_extended["is_boosted"]=leboncoin_ads_filtered_extended["is_boosted"].apply(lambda x: 0 if math.isnan(x) else x)
cols=["images","profile_picture_url","vehicle_is_eligible_p2p","car_rotation_delay","licence_plate_available",
      "vehicle_history_report_public_url","vehicle_damage","activity_sector","vehicle_history_report_status"]
leboncoin_ads_filtered_extended[cols]=leboncoin_ads_filtered_extended[cols].apply(lambda col: col.map(lambda x: 0 if pd.isna(x) else 1))
# convertir en numérique
leboncoin_ads_filtered_extended["mileage"]=leboncoin_ads_filtered_extended["mileage"].apply(lambda x: float(x.split(" km")[0]))
cols=['rating_score', 'car_price_min', 'car_price_max',"old_price","siren"]
leboncoin_ads_filtered_extended[cols]=leboncoin_ads_filtered_extended[cols].apply(lambda col: col.map(lambda x: float(x) if pd.notna(x) else np.nan))
#replace Nan par "non spécifé"
cols=["doors","seats","vehicle_technical_inspection_a","vehicle_type","vehicle_upholstery","vehicle_specifications",
      "vehicule_color","critair","horsepower","horse_power_din","ad_warranty_type","recent_used_vehicle","vehicle_history_report_status",
        "spare_parts_availability","vehicle_euro_emissions_standard"]
leboncoin_ads_filtered_extended[cols]=leboncoin_ads_filtered_extended[cols].apply(lambda col: col.map (lambda x: "unknown" if pd.isna(x) else x))
leboncoin_ads_filtered_extended["vehicle_interior_specs"]=leboncoin_ads_filtered_extended["vehicle_interior_specs"].apply(lambda x: "unknow interior" if pd.isna(x) else x)
#renommer certaines modalités:
leboncoin_ads_filtered_extended["vehicle_interior_specs"]=leboncoin_ads_filtered_extended["vehicle_interior_specs"].apply(lambda l: [ 
        'Phares LED' if x == 'Phares LED / Xenon' 
        else 'Toit ouvrant' if x == 'Toit ouvrant / Toit panoramique' 
        else x
        for x in l.split(", ")] if isinstance(l,list) else l
)

In [12]:
#More shattering
for spec in ["Climatisation", "Toit ouvrant",  "Régulateur de vitesse", "Système de navigation", "Attelage de remorque", 
             "Aide au stationnement", "Jantes en alliage", "Bluetooth", "Phares LED", "Sièges chauffants","unknow interior"]:
    leboncoin_ads_filtered_extended[spec]=leboncoin_ads_filtered_extended['vehicle_interior_specs'].apply(lambda x: 1 if spec in x else 0)

In [13]:
# Enlever les variables contenant les dictionnaires
leboncoin_ads_filtered_extended=leboncoin_ads_filtered_extended.drop(columns=["attributes",
                                                                              "location",
                                                                              "owner",
                                                                              "options","vehicle_interior_specs"])
#Enlever variables répétitives:
leboncoin_ads_filtered_extended=leboncoin_ads_filtered_extended.drop(columns=["brand",
                                                                              "model",
                                                                              "rating_count",
                                                                              "region_id",
                                                                              "department_id",
                                                                              "city_label"
                                                                            ])
#Enlever variables non pertinentes:
leboncoin_ads_filtered_extended=leboncoin_ads_filtered_extended.drop(columns=["images",
                                                                              "argus_object_id",
                                                                              "feature"
                                                                                ])
#Enlever variables avec une seule modalité:
leboncoin_ads_filtered_extended=leboncoin_ads_filtered_extended.drop(columns=["vehicle_vsp",
                                                                              "is_import",
                                                                              "country_id",
                                                                              "provider","booster",
                                                                              'photosup', 'urgent', 'gallery',
                                                                              'continuous_top_ads','highlight'
                                                                                ])

Rename columns for clarity

In [21]:
leboncoin_ads_filtered_extended=leboncoin_ads_filtered_extended.rename(columns={"mileage":"mileage_km",
                                                                                "horsepower":"horsepower_cv"})

In [22]:
print("Taille:",leboncoin_ads_filtered_extended.shape,"\ncolumns: ",leboncoin_ads_filtered_extended.columns)

Taille: (35, 68) 
columns:  Index(['list_id', 'first_publication_date', 'expiration_date', 'ad_type',
       'price', 'has_phone', 'is_boosted', 'profile_picture_url',
       'u_car_brand', 'u_car_model', 'regdate', 'mileage_km', 'fuel',
       'gearbox', 'doors', 'seats', 'vehicle_technical_inspection_a',
       'issuance_date', 'vehicle_damage', 'vehicle_type', 'vehicle_upholstery',
       'vehicle_specifications', 'vehicule_color', 'critair', 'horsepower_cv',
       'horse_power_din', 'vehicle_is_eligible_p2p', 'car_rotation_delay',
       'licence_plate_available', 'rating_score', 'ad_warranty_type',
       'car_price_min', 'car_price_max', 'vehicle_history_report_public_url',
       'vehicle_euro_emissions_standard', 'activity_sector',
       'recent_used_vehicle', 'vehicle_history_report_status',
       'spare_parts_availability', 'car_contract', 'old_price', 'region_name',
       'department_name', 'city', 'zipcode', 'lat', 'lng', 'source',
       'is_shape', 'store_id', 'user_i